In [1]:
import numpy as np
import cv2 as cv
import os
import math
from scipy.ndimage import rotate

In [2]:
tabla =[
    [5,0,0,4,0,0,0,3,0,0,0,4,0,0,5],
    [0,0,3,0,0,4,0,0,0,4,0,0,3,0,0],
    [0,3,0,0,2,0,0,0,0,0,2,0,0,3,0],
    [4,0,0,3,0,2,0,0,0,2,0,3,0,0,4],
    [0,0,2,0,1,0,1,0,1,0,1,0,2,0,0],
    [0,4,0,2,0,1,0,0,0,1,0,2,0,4,0],
    [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
    [3,0,0,0,0,0,0,0,0,0,0,0,0,0,3],
    [0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
    [0,4,0,2,0,1,0,0,0,1,0,2,0,4,0],
    [0,0,2,0,1,0,1,0,1,0,1,0,2,0,0],
    [4,0,0,3,0,2,0,0,0,2,0,3,0,0,4],
    [0,3,0,0,2,0,0,0,0,0,2,0,0,3,0],
    [0,0,3,0,0,4,0,0,0,4,0,0,3,0,0],
    [5,0,0,4,0,0,0,3,0,0,0,4,0,0,5]
]


traseu_scor =[7,1,2,3,4,5,6,0,2,5,3,4,6,2,2,0,3,5,4,1,6,2,4,5,5,0,6,3,4,2,0,1,5,1,3,4,4,4,5,0,6,3,5,4,1,3,2,0,0,1,1,2,3,6,3,5,2,1,0,6,6,5,2,1,2,5,0,3,3,5,0,6,1,4,0,6,3,5,1,4,2,6,2,3,1,6,5,6,2,0,4,0,1,6,4,4,1,6,6,3,100]


In [3]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.5,fy=0.5)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [4]:
def mask_careu(image):
    img_copy = image.copy()  
    img_copy = cv.medianBlur(img_copy,5)
    img_copy = cv.GaussianBlur(img_copy, (5, 5), 255)
    kernel = np.ones((5, 5), np.uint8) 
    hsv = cv.cvtColor(img_copy, cv.COLOR_BGR2HSV) 
    lower_blue = np.array([15, 150, 55]) 
    upper_blue = np.array([175, 255, 255]) 
    Mask = cv.inRange(hsv, lower_blue, upper_blue) 
    Mask = cv.dilate(Mask, kernel, iterations=1) 
    Mask = cv.bitwise_not(Mask) 
    Mask = cv.medianBlur(Mask,9)
    Mask = cv.GaussianBlur(Mask, (5,5), 0)
    cv.addWeighted(Mask, 2.0, Mask, -0, 0, Mask)
    edges =  cv.Canny(Mask,200,400)
    
    return edges

In [5]:
def edit_piesa(image):
    
    kernel = np.ones((5,5), np.uint8)
    edges = cv.erode(image, kernel)
    
    return edges

In [6]:
def extrage_careu_piesa(image):
    
    img_copy = image.copy() 
    contours, _ = cv.findContours(img_copy,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    return top_left,top_right,bottom_left,bottom_right

In [7]:
def diff_imagini(poz1,poz2): 
    
    kernel = np.ones((5,5), np.uint8)
    
    poz1 = cv.cvtColor(poz1,cv.COLOR_BGR2GRAY)
    cv.addWeighted(poz1, 0.8, poz1, -0.3, 0, poz1)
    _, thresh1 = cv.threshold(poz1, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh1 = cv.erode(thresh1, kernel)
    
    poz2 = cv.cvtColor(poz2,cv.COLOR_BGR2GRAY)
    cv.addWeighted(poz2, 0.8, poz2, -0.3, 0, poz2)
    _, thresh2 = cv.threshold(poz2, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh2 = cv.erode(thresh2, kernel)
    
    diff_img = abs(thresh2-thresh1)
    #show_image('dif',diff_img)
    return diff_img

In [8]:
def poz_piesa(piesa):
    
    piesa = edit_piesa(piesa)
    top_left,top_right,bottom_left,bottom_right = extrage_careu_piesa(piesa)
    
    x=math.floor((top_left[1] + height_patrat/2) / height_patrat) + 1
    y=math.floor((top_left[0] + width_patrat/2) / width_patrat) + 1
  
    top_len = math.dist(top_left,top_right)
    
    left_len = math.dist(bottom_left,top_left)
    
    right_len = math.dist(top_right,bottom_right)
    
    bottom_len = math.dist(bottom_left,bottom_right)
    piesa1 = [x,y]

    if top_len > left_len or bottom_len > right_len:
        orizontal = True
    else:
        orizontal=False
    
    if orizontal:
        piesa2=[x,y+1]
    else:
        piesa2=[x+1,y]
    
    return piesa1,piesa2,orizontal

In [9]:
def clasifica_piesa(patch,orizontal):
        maxi=-np.inf
        poz=-1
        folder_template1 = 'poze_piese_vert/'
        folder_template2 = 'poze_piese_orizont/'
        files1 = os.listdir(folder_template1)
        files2 = os.listdir(folder_template2)
        patch = cv.cvtColor(patch,cv.COLOR_BGR2GRAY)
        _, patch = cv.threshold(patch, 125, 255, cv.THRESH_BINARY)
        
        if orizontal:
            for file in files2:
                img_template = cv.imread(folder_template2+file)
                img_template = cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
                _, img_template = cv.threshold(img_template, 125, 255, cv.THRESH_BINARY)  

                corr = cv.matchTemplate(patch,img_template,  cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=file
                    rotated=False
        
                rotate_img = rotate(img_template, angle=180) 
                corr = cv.matchTemplate(patch,rotate_img,  cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=file
                    rotated = True
        else:
            for file in files1:
                img_template = cv.imread(folder_template1+file)
                img_template = cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
                _, img_template = cv.threshold(img_template, 125, 255, cv.THRESH_BINARY)  
                corr = cv.matchTemplate(patch,img_template,  cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=file
                    rotated = False
        
                rotate_img = rotate(img_template, angle=180) 
                corr = cv.matchTemplate(patch,rotate_img,  cv.TM_CCOEFF_NORMED)
                corr = np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=file
                    rotated = True
              
        return poz, rotated

In [11]:
folder_input = 'antrenare/'
folder_output = 'Rezultate/'
empty_board = 'imagini_auxiliare/01.jpg'
files = os.listdir(folder_input)

mutari_totale = np.zeros(21)
for file in files:
    if file[-3:]=='jpg':
        
        img = cv.imread(folder_input+file)
        split = file.split('_')
        nr_joc = split[0]
        mutarea_curenta = split[1][:2]

        with open(folder_input+nr_joc+'_mutari.txt') as f:
            lines = f.readlines()  
        jucator = int(lines[int(mutarea_curenta)-1][-2])
        
        nr_joc = int(nr_joc)
        if mutari_totale[nr_joc] == 0:
            
            score_board = np.zeros(2)
            mutari_totale[nr_joc] = nr_joc
            
            img1 = mask_careu(img)
            top_left,top_right,bottom_left,bottom_right = extrage_careu_piesa(img1)
            
            #latimea si inaltimea (aproximative) a unui patrat din careul extras
            height_patrat = round((bottom_right[1]-top_left[1])/15)
            width_patrat = round((bottom_right[0]-top_left[0])/15)

        careu_curent = img[int(top_left[1]):int(bottom_right[1]+20),
                              int(top_left[0]):int(bottom_right[0])+20]
        
        marime_careu = careu_curent.shape
        
        mutarea_curenta = int(mutarea_curenta)
        if mutarea_curenta == 1:
            
            prev_img = cv.imread(empty_board)            
            img1= mask_careu(prev_img)
            top_left_new,top_right_new,bottom_left_new,bottom_right_new = extrage_careu_piesa(img1)
            
            careu_anterior = prev_img[int(top_left_new[1]):int(bottom_right_new[1])+20,
                            int(top_left_new[0]):int(bottom_right_new[0])+20]
        
            careu_anterior = cv.resize(careu_anterior,(marime_careu[1],marime_careu[0]))
            
        else:
            careu_anterior = prev_img[int(top_left[1]):int(bottom_right[1])+20,
                            int(top_left[0]):int(bottom_right[0])+20]
            

            

        diff_img = diff_imagini(careu_anterior,careu_curent)
        diff_img=cv.erode(diff_img,kernel=np.ones((6,6), np.uint8), iterations=1)
        
        cv.addWeighted(diff_img, 0.5, diff_img, 0, 0, diff_img)
        
        diff_img=cv.dilate(diff_img,kernel=np.ones((5,5), np.uint8), iterations=4)
        diff_img=cv.erode(diff_img,kernel=np.ones((4,4), np.uint8), iterations=1)
        diff_img=cv.dilate(diff_img,kernel=np.ones((5,5), np.uint8), iterations=5)

        # print("\n\n",mutarea_curenta," : ",poz_piesa(diff_img))

        poz_piesa1, poz_piesa2, orizontal = poz_piesa(diff_img)

        coord_top_left = [(poz_piesa1[1]-1) * width_patrat,
                          (poz_piesa1[0]-1) * height_patrat]

        if orizontal:
            coord_bottom_right = [coord_top_left[0] + 2 * width_patrat,
                                  coord_top_left[1] + height_patrat]
        else:
            coord_bottom_right = [coord_top_left[0] + width_patrat,
                                  coord_top_left[1] + 2 *height_patrat]

        temp_img = careu_curent.copy()
        temp_img = cv.copyMakeBorder(temp_img, 10,10,10,10, borderType=cv.BORDER_CONSTANT,dst=None,value=[0,0,255])

        
        piece_img = careu_curent[coord_top_left[1]:coord_bottom_right[1] + 20,
                    coord_top_left[0]:coord_bottom_right[0] + 20]
        #show_image('piesa', piece_img)
        piesa,rotire = clasifica_piesa(piece_img,orizontal)

        sir=  piesa.split('_')
        p1=int(piesa[0])
        p2=int(piesa[2])
        
        if rotire:
            aux=p1
            p1=p2
            p2=aux
            
        prev_img = img
        
        score1 = tabla[poz_piesa1[0]-1][poz_piesa1[1]-1]
        score2 = tabla[poz_piesa2[0]-1][poz_piesa2[1]-1]
        scor_runda = 0

        for index, pion in enumerate(score_board):
                pion = int(pion)
                if traseu_scor[pion] == int(p1) or traseu_scor[pion] == int(p2):
                    score_board[index]+=3  
                    if id+1 == jucator:
                        scor_runda+=3

        if score1 != 0:
            if p1 == p2:
                score1 = score1 * 2
            score_board[jucator-1] += score1
            scor_runda += score1

        elif score2 != 0:
            if p1 == p2:
                score2 = score2 * 2
            score_board[jucator-1] += score2
            scor_runda += score2

        
        with open(folder_output+file[:-4]+'.txt', 'w') as f:
            f.write(str(poz_piesa1[0])+str(chr(64 + poz_piesa1[1]))+" "+str(p1))
            f.write('\n')
            f.write(str(poz_piesa2[0])+str(chr(64 + poz_piesa2[1]))+" "+str(p2))
            f.write('\n')
            f.write(str(scor_runda))

cv.waitKey(0)
cv.destroyAllWindows()